In [1]:
import sys
sys.path.append("/home/lauren/Pufferfish")

In [3]:
%cd ..

/home/yongqin/train_imagenet/Pufferfish


/home/yongqin/myPython3/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
from resnet_cifar10 import *
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import torch
from torch import no_grad, max, device, load
import torchvision.transforms as transforms


In [7]:
%cd checkpoint

/home/yongqin/train_imagenet/Pufferfish/checkpoint


/home/yongqin/myPython3/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [10]:
model = LowRankWideResNet18()
weights = "wideresnet18_lowrank_300epochs_rf2.pth"
checkpoint = load(weights, map_location=torch.device('cuda'))
model.load_state_dict(checkpoint['net'])

<All keys matched successfully>

In [11]:
model.eval()
device = torch.device('cuda')
model.to(device)

HybridWideResNet(
  (conv1): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): LowrankBasicBlock(
      (conv1_u): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv1_v): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2_u): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2_v): Conv2d(64, 128, kernel

In [12]:

normalize = transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]],
                                std=[x/255.0 for x in [63.0, 62.1, 66.7]])
transform_test = transforms.Compose([
        transforms.ToTensor(),
        normalize])

testset = CIFAR10(root='./data', train=False, download=True, transform=transform_test)

testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

100.0%


In [13]:
dataiter = iter(testloader)
images, labels = next(dataiter)

print(images.shape)
print(labels.shape)

torch.Size([100, 3, 32, 32])
torch.Size([100])


In [14]:
correct = 0
total = 0
with no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = max(outputs, 1)
        total+=labels.size(0)
        correct += (predicted == labels).sum().item()

In [15]:
print(correct/total)

0.9444


In [33]:
model2 = LowRankWideResNet18()
weights = "wideresnet18_lowrank_300epochs.pth"
checkpoint = load(weights, map_location=torch.device('cuda'))
model2.load_state_dict(checkpoint['net'])
model2.eval()
device = torch.device('cuda')
model2.to(device)

/tmp/ipykernel_210833/543645346.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = load(weights, map_location=torch.device('cuda'))


HybridWideResNet(
  (conv1): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): LowrankBasicBlock(
      (conv1_u): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv1_v): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2_u): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2_v): Conv2d(32, 128, kernel

In [34]:
correct2 = 0
total2 = 0
with no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs2 = model2(images)
        _, predicted = max(outputs2, 1)
        total2+=labels.size(0)
        correct2 += (predicted == labels).sum().item()

In [35]:
print(correct2/total2)

0.9424
